In [78]:
import xarray as xr
import pandas as pd
import locale

locale.setlocale(locale.LC_ALL,'en_US')
dataset = xr.open_dataset('MiningProcess_Flotation_Plant_Database.h5', engine = 'h5netcdf')
df = dataset.to_pandas()


In [79]:
## Resampling
df = df.resample('1h').mean()

## Split into input and output
X = df.iloc[:,:-2]
Y = df['% Silica Concentrate']

In [80]:
# Adding lagged variables

X_copy = X.copy()

import numpy as np


for variable in X_copy.columns:
    # print(variable)
    
    X_variable = X_copy[variable]

    for lag in range(1,6):
        lagged_variable = np.zeros(len(X))
        lagged_variable[lag:] = X_variable.iloc[:-lag]

    X.loc[:,f'{variable}, Lag {lag}'] = lagged_variable

X = X[5:]

C:\Users\antto\AppData\Local\Temp\ipykernel_2324\3760031147.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,f'{variable}, Lag {lag}'] = lagged_variable


In [81]:
# Split to calibration-validation set and test set
X_cal = X['20170615':'20170723']
Y_cal = Y['20170615':'20170723']
X_test = X['20170412':'20170506']
Y_test = Y['20170412':'20170506']

In [82]:
# Remove missing values in between
X_test.loc['20170421':'20170424'] = pd.NA
X_test.dropna(inplace=True)

Y_test.loc['20170421':'20170424'] = pd.NA
Y_test.dropna(inplace=True)

C:\Users\antto\AppData\Local\Temp\ipykernel_2324\2442657920.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.dropna(inplace=True)


In [83]:
# Store data

with pd.HDFStore('MiningProcess_Flotation_Plant_Database_train_split.h5') as store:
    store.put('X_cal', X_cal)
    store.put('Y_cal', Y_cal)
    store.put('X_test', X_test)
    store.put('Y_test', Y_test)